<a href="https://colab.research.google.com/github/dubeyji10/techgig-SBI-challenge/blob/main/azure_ai_FORM_RECOGNIZER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

''' pip install azure-ai-formrecognizer==3.2.0b3 '''

In [ ]:
print('connected')

connected


In [ ]:
!pip install azure-ai-formrecognizer==3.2.0b3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 246 kB 9.8 MB/s 
     |████████████████████████████████| 85 kB 4.0 MB/s 
     |████████████████████████████████| 178 kB 42.5 MB/s 
     |████████████████████████████████| 41 kB 545 kB/s 


In [ ]:
!pip list

Package                       Version
----------------------------- ------------------------------
absl-py                       1.0.0
alabaster                     0.7.12
albumentations                0.1.12
altair                        4.2.0
appdirs                       1.4.4
argon2-cffi                   21.3.0
argon2-cffi-bindings          21.2.0
arviz                         0.12.1
astor                         0.8.1
astropy                       4.3.1
astunparse                    1.6.3
atari-py                      0.2.9
atomicwrites                  1.4.0
attrs                         21.4.0
audioread                     2.1.9
autograd                      1.4
azure-ai-formrecognizer       3.2.0b3
azure-common                  1.1.28
azure-core                    1.24.1
Babel                         2.10.1
backcall                      0.2.0
beautifulsoup4                4.6.3
bleach                        5.0.0
blis                          0.4.1
bokeh                       

# trying to extract text from pdf

### using azure ocr-form

In [ ]:
# import libraries
import os
from azure.ai.formrecognizer import DocumentAnalysisClient
from azure.core.credentials import AzureKeyCredential

# set `<your-endpoint>` and `<your-key>` variables with the values from the Azure portal

def format_bounding_region(bounding_regions):
    if not bounding_regions:
        return "N/A"
    return ", ".join("Page #{}: {}".format(region.page_number, format_bounding_box(region.bounding_box)) for region in bounding_regions)

def format_bounding_box(bounding_box):
    if not bounding_box:
        return "N/A"
    return ", ".join(["[{}, {}]".format(p.x, p.y) for p in bounding_box])


def analyze_general_documents(docUrl):
    # sample document
    # docUrl = "https://raw.githubusercontent.com/Azure-Samples/cognitive-services-REST-api-samples/master/curl/form-recognizer/sample-layout.pdf"

    # create your `DocumentAnalysisClient` instance and `AzureKeyCredential` variable
    document_analysis_client = DocumentAnalysisClient(endpoint=endpoint, credential=AzureKeyCredential(key))

    poller = document_analysis_client.begin_analyze_document_from_url(
            "prebuilt-document", docUrl)
    result = poller.result()

    for style in result.styles:
        if style.is_handwritten:
            print("Document contains handwritten content: ")
            print(",".join([result.content[span.offset:span.offset + span.length] for span in style.spans]))

    print("----Key-value pairs found in document----")
    for kv_pair in result.key_value_pairs:
        if kv_pair.key:
            print(
                    "Key '{}' found within '{}' bounding regions".format(
                        kv_pair.key.content,
                        format_bounding_region(kv_pair.key.bounding_regions),
                    )
                )
        if kv_pair.value:
            print(
                    "Value '{}' found within '{}' bounding regions\n".format(
                        kv_pair.value.content,
                        format_bounding_region(kv_pair.value.bounding_regions),
                    )
                )

    print("----Entities found in document----")
    for entity in result.entities:
        print("Entity of category '{}' with sub-category '{}'".format(entity.category, entity.sub_category))
        print("...has content '{}'".format(entity.content))
        print("...within '{}' bounding regions".format(format_bounding_region(entity.bounding_regions)))
        print("...with confidence {}\n".format(entity.confidence))

    for page in result.pages:
        print("----Analyzing document from page #{}----".format(page.page_number))
        print(
            "Page has width: {} and height: {}, measured with unit: {}".format(
                page.width, page.height, page.unit
            )
        )

        for line_idx, line in enumerate(page.lines):
            print(
                "...Line # {} has text content '{}' within bounding box '{}'".format(
                    line_idx,
                    line.content,
                    format_bounding_box(line.bounding_box),
                )
            )

        for word in page.words:
            print(
                "...Word '{}' has a confidence of {}".format(
                    word.content, word.confidence
                )
            )

        for selection_mark in page.selection_marks:
            print(
                "...Selection mark is '{}' within bounding box '{}' and has a confidence of {}".format(
                    selection_mark.state,
                    format_bounding_box(selection_mark.bounding_box),
                    selection_mark.confidence,
                )
            )

    for table_idx, table in enumerate(result.tables):
        print(
            "Table # {} has {} rows and {} columns".format(
                table_idx, table.row_count, table.column_count
            )
        )
        for region in table.bounding_regions:
            print(
                "Table # {} location on page: {} is {}".format(
                    table_idx,
                    region.page_number,
                    format_bounding_box(region.bounding_box),
                )
            )
        for cell in table.cells:
            print(
                "...Cell[{}][{}] has content '{}'".format(
                    cell.row_index,
                    cell.column_index,
                    cell.content,

                )
            )
            for region in cell.bounding_regions:
                print(
                    "...content on page {} is within bounding box '{}'\n".format(
                        region.page_number,
                        format_bounding_box(region.bounding_box),
                    )
                )
    print("----------------------------------------")


### testing on image ( from internet )
<img src="https://i.postimg.cc/v8XrHPRX/sbi-rtgs-sample.jpg">

In [ ]:
%%time

# analyze_general_documents()

myDoc = "https://i.postimg.cc/v8XrHPRX/sbi-rtgs-sample.jpg"
analyze_general_documents(myDoc)


Document contains handwritten content: 
₹(in words) Twelve thousand Only,33804197263
20,Aman Yadav,₹ 12000 00 60,RDSO
13427,Branch
3380419726320,12000,Rohit
Document contains handwritten content: 
₹
Document contains handwritten content: 
SBI
Document contains handwritten content: 
SBINOO-
----Key-value pairs found in document----
Key 'Account No. for RTGS/NEFT/ खाता की पुष्टि करें /' found within 'Page #1: [127.0, 188.0], [353.0, 187.0], [354.0, 384.0], [128.0, 385.0]' bounding regions
Value '33804197263' found within 'Page #1: [21.0, 224.0], [425.0, 223.0], [427.0, 252.0], [23.0, 253.0]' bounding regions

Key 'Name of Beneficiary' found within 'Page #1: [357.0, 187.0], [521.0, 187.0], [521.0, 208.0], [357.0, 208.0]' bounding regions
Value '20 Aman Yadav' found within 'Page #1: [141.0, 232.0], [515.0, 226.0], [516.0, 301.0], [143.0, 306.0]' bounding regions

Key 'Amount' found within 'Page #1: [1017.0, 184.0], [1099.0, 183.0], [1099.0, 201.0], [1018.0, 202.0]' bounding regions
Value '

In [ ]:

def format_bounding_region(bounding_regions):
    if not bounding_regions:
        return "N/A"
    return ", ".join("Page #{}: {}".format(region.page_number, format_bounding_box(region.bounding_box)) for region in bounding_regions)

def format_bounding_box(bounding_box):
    if not bounding_box:
        return "N/A"
    return ", ".join(["[{}, {}]".format(p.x, p.y) for p in bounding_box])


def analyze_general_documents2(docUrl):
    # sample document
    # docUrl = "https://raw.githubusercontent.com/Azure-Samples/cognitive-services-REST-api-samples/master/curl/form-recognizer/sample-layout.pdf"

    # create your `DocumentAnalysisClient` instance and `AzureKeyCredential` variable
    document_analysis_client = DocumentAnalysisClient(endpoint=endpoint, credential=AzureKeyCredential(key))

    poller = document_analysis_client.begin_analyze_document_from_url(
            "prebuilt-document", docUrl)
    result = poller.result()

    for style in result.styles:
        if style.is_handwritten:
            print("Document contains handwritten content: ")
            print(",".join([result.content[span.offset:span.offset + span.length] for span in style.spans]))

    print("----Key-value pairs found in document----")
    for kv_pair in result.key_value_pairs:
        if kv_pair.key:
            print(
                    "Key '{}' found within '{}' bounding regions".format(
                        kv_pair.key.content,
                        format_bounding_region(kv_pair.key.bounding_regions),
                    )
                )
        if kv_pair.value:
            print(
                    "Value '{}' found within '{}' bounding regions\n".format(
                        kv_pair.value.content,
                        format_bounding_region(kv_pair.value.bounding_regions),
                    )
                )

    print("----Entities found in document----")
    for entity in result.entities:
        # print("Entity of category '{}' with sub-category '{}'".format(entity.category, entity.sub_category))
        print("...has content '{}'".format(entity.content))
        # print("...within '{}' bounding regions".format(format_bounding_region(entity.bounding_regions)))
        # print("...with confidence {}\n".format(entity.confidence))

    for page in result.pages:
        print("----Analyzing document from page #{}----".format(page.page_number))
        print(
            "Page has width: {} and height: {}, measured with unit: {}".format(
                page.width, page.height, page.unit
            )
        )

        for line_idx, line in enumerate(page.lines):
            print(
                "...Line # {} has text content '{}' within bounding box '{}'".format(
                    line_idx,
                    line.content,
                    format_bounding_box(line.bounding_box),
                )
            )

        for word in page.words:
            print(
                "...Word '{}' has a confidence of {}".format(
                    word.content, word.confidence
                )
            )

        for selection_mark in page.selection_marks:
            print(
                "...Selection mark is '{}' within bounding box '{}' and has a confidence of {}".format(
                    selection_mark.state,
                    format_bounding_box(selection_mark.bounding_box),
                    selection_mark.confidence,
                )
            )

    for table_idx, table in enumerate(result.tables):
        print(
            "Table # {} has {} rows and {} columns".format(
                table_idx, table.row_count, table.column_count
            )
        )
        for region in table.bounding_regions:
            print(
                "Table # {} location on page: {} is {}".format(
                    table_idx,
                    region.page_number,
                    format_bounding_box(region.bounding_box),
                )
            )
        for cell in table.cells:
            print(
                "...Cell[{}][{}] has content '{}'".format(
                    cell.row_index,
                    cell.column_index,
                    cell.content,

                )
            )
            for region in cell.bounding_regions:
                print(
                    "...content on page {} is within bounding box '{}'\n".format(
                        region.page_number,
                        format_bounding_box(region.bounding_box),
                    )
                )
    print("----------------------------------------")


### testing on image ( from internet )
<img src="https://i.postimg.cc/v8XrHPRX/sbi-rtgs-sample.jpg">


In [ ]:

%%time

# analyze_general_documents()

myDoc = "https://i.postimg.cc/v8XrHPRX/sbi-rtgs-sample.jpg"
analyze_general_documents2(myDoc)


Document contains handwritten content: 
₹(in words) Twelve thousand Only,33804197263
20,Aman Yadav,₹ 12000 00 60,RDSO
13427,Branch
3380419726320,12000,Rohit
Document contains handwritten content: 
₹
Document contains handwritten content: 
SBI
Document contains handwritten content: 
SBINOO-
----Key-value pairs found in document----
Key 'Account No. for RTGS/NEFT/ खाता की पुष्टि करें /' found within 'Page #1: [127.0, 188.0], [353.0, 187.0], [354.0, 384.0], [128.0, 385.0]' bounding regions
Value '33804197263' found within 'Page #1: [21.0, 224.0], [425.0, 223.0], [427.0, 252.0], [23.0, 253.0]' bounding regions

Key 'Name of Beneficiary' found within 'Page #1: [357.0, 187.0], [521.0, 187.0], [521.0, 208.0], [357.0, 208.0]' bounding regions
Value '20 Aman Yadav' found within 'Page #1: [141.0, 232.0], [515.0, 226.0], [516.0, 301.0], [143.0, 306.0]' bounding regions

Key 'Amount' found within 'Page #1: [1017.0, 184.0], [1099.0, 183.0], [1099.0, 201.0], [1018.0, 202.0]' bounding regions
Value '